In [60]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization


In [61]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset ='/home/turning/Desktop/CL_project/CL_PROJECT_CODE/Kerasmodel/aclImdb'

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

['README', 'train']

In [62]:
train_dir = '/home/turning/Desktop/CL_project/CL_PROJECT_CODE/Kerasmodel/aclImdb/train/'
os.listdir(train_dir)

['pos']

In [63]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/home/turning/Desktop/CL_project/CL_PROJECT_CODE/Kerasmodel/aclImdb/train/unsup'

In [64]:
batch_size = 1024
seed = 123
train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

Found 6 files belonging to 1 classes.
Using 5 files for training.
Found 6 files belonging to 1 classes.
Using 1 files for validation.


In [67]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [68]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(10000, 5)

In [69]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[-0.04833598, -0.00230513,  0.00545232, -0.00936279,  0.03290052],
       [ 0.04968916,  0.03968636, -0.04100504, -0.00653931, -0.02838165],
       [ 0.02774164, -0.02049837, -0.0340514 , -0.041743  ,  0.00354023]],
      dtype=float32)

In [70]:
result = embedding_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))
result.shape

TensorShape([2, 3, 5])

In [71]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [72]:
embedding_dim=20

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

In [73]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [74]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [75]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
1/1 [==============================] - 1s 617ms/step - loss: 0.6926 - accuracy: 1.0000 - val_loss: 0.6917 - val_accuracy: 1.0000
Epoch 2/15
1/1 [==============================] - 0s 41ms/step - loss: 0.6891 - accuracy: 1.0000 - val_loss: 0.6896 - val_accuracy: 1.0000
Epoch 3/15
1/1 [==============================] - 0s 38ms/step - loss: 0.6859 - accuracy: 1.0000 - val_loss: 0.6872 - val_accuracy: 1.0000
Epoch 4/15
1/1 [==============================] - 0s 40ms/step - loss: 0.6828 - accuracy: 1.0000 - val_loss: 0.6848 - val_accuracy: 1.0000
Epoch 5/15
1/1 [==============================] - 0s 39ms/step - loss: 0.6795 - accuracy: 1.0000 - val_loss: 0.6824 - val_accuracy: 1.0000
Epoch 6/15
1/1 [==============================] - 0s 41ms/step - loss: 0.6761 - accuracy: 1.0000 - val_loss: 0.6800 - val_accuracy: 1.0000
Epoch 7/15
1/1 [==============================] - 0s 42ms/step - loss: 0.6727 - accuracy: 1.0000 - val_loss: 0.6775 - val_accuracy: 1.0000
Epoch 8/15
1/1 [==========

In [76]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_3 (TextV  (None, 100)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 100, 20)           200000    
                                                                 
 global_average_pooling1d_4   (None, 20)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_8 (Dense)             (None, 16)                336       
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 200,353
Trainable params: 200,353
Non-tr

In [77]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 10542), started 0:28:40 ago. (Use '!kill 10542' to kill it.)

In [ ]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass